In [2]:
import shapely
from shapely import Point
import numpy as np
import regex as re

In [1]:
file1 = open('input.txt', 'r')
lines = file1.readlines()

In [30]:
rules = {'A': ["A"], 'R': ["R"]}
ranges = []

class Rule:
    def __init__(self, x_range, m_range, a_range, s_range, outcome):
        self.x_range = x_range
        self.m_range = m_range
        self.a_range = a_range
        self.s_range = s_range
        self.outcome = outcome

    def get_outcome(self, x, m, a, s):
        if (x >= self.x_range[0] and x <= self.x_range[1]
            and m >= self.m_range[0] and m <= self.m_range[1]
            and a >= self.a_range[0] and a <= self.a_range[1]
            and s >= self.s_range[0] and s <= self.s_range[1]):
                return self.outcome
        return None

def range_finder(current_rule, id, current_ranges):
    if current_rule[id] == 'R':
        ranges.append(Rule(current_ranges["x"], current_ranges["m"], current_ranges["a"], current_ranges["s"], "R"))
        return None
    if current_rule[id] == 'A':
        ranges.append(Rule(current_ranges["x"], current_ranges["m"], current_ranges["a"], current_ranges["s"], "A"))
        return None
    print(current_rule[id])
    quoted_string = re.match(r'([a|s|x|m])([<|>])(\d+):(\w+)', current_rule[id]).groups()
    letter = quoted_string[0]
    sign = quoted_string[1]
    number = int(quoted_string[2])
    next_rule = quoted_string[3]
    if sign == '<':
        if current_ranges[letter][1] < number:
            range_finder(rules[next_rule], 0, current_ranges)
        elif current_ranges[letter][0] > number:
            range_finder(current_rule, id+1, current_ranges)
        else:
            bottom_range = current_ranges.copy()
            top_range = current_ranges.copy()
            bottom_range[letter][1] = number -1
            top_range[letter][0] = number
            range_finder(rules[next_rule], 0, bottom_range)
            range_finder(current_rule, id+1, top_range)
    elif sign == '>':
        if current_ranges[letter][0] > number:
            range_finder(rules[next_rule], 0, current_ranges)
        elif current_ranges[letter][1] < number:
            range_finder(current_rule, id+1, current_ranges)
        else:
            bottom_range = current_ranges.copy()
            top_range = current_ranges.copy()
            bottom_range[letter][1] = number
            top_range[letter][0] = number + 1
            range_finder(rules[next_rule], 0, top_range)
            range_finder(current_rule, id+1, bottom_range)
    print("Error")

for line in lines:
    if line == '\n':
        current_ranges = {"x": [0, 9999], "m": [0, 9999], "a": [0, 9999], "s": [0, 9999]}
        range_finder(rules[list(rules.keys())[2]], 0, current_ranges)
    parsed_string = re.match(r'(\w+)\{(.*)\}', line)
    if parsed_string:
        rules[parsed_string.groups()[0]] = [i for i in parsed_string.groups()[1].split(',')]



a<2006:qkq
x<1416:A
crn


AttributeError: 'NoneType' object has no attribute 'groups'

In [10]:
re.match(r'(\w+)\{(.*)\}', lines[11]).groups()

AttributeError: 'NoneType' object has no attribute 'groups'

In [12]:
re.match(r'(\w+)\{(.*)\}', lines[1])

<regex.Match object; span=(0, 14), match='pv{a>1716:R,A}'>